# Keras Digit Recogniser

In [55]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [56]:
# The code was removed by Watson Studio for sharing.

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [57]:
# preview data
train.head()
#test.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
labels = train.label.values.astype('int32')
train_img = train.loc[:,'pixel0':].values.astype('float32')
#labels
train_img.shape, test.shape
#type(labels)

((42000, 784), (28000, 784))

In [59]:
# convert labels to a categorical data type, as expected by keras model
labels = to_categorical(labels)
classes = labels.shape[1]
labels

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [60]:
# fix random seed for reproducibility
seed = 43
np.random.seed(seed)

## Neural Network

Now we will build the neural network. We use a Keras Sequential model (i.e. simple feedforward network), with Relu input and hidden layer activations, and a softmax output layer with 10 classes. model.add() will be used to add layers.

In [61]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [62]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [63]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model_fit = model.fit(train_img, labels, epochs=100, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 [==============================] - 8s 211us/step - loss: 0.6879 - acc: 0.7788 - val_loss: 0.3244 - val_acc: 0.9093
Epoch 2/100
37800/37800 [==============================] - 8s 200us/step - loss: 0.2691 - acc: 0.9240 - val_loss: 0.2449 - val_acc: 0.9340
Epoch 3/100
37800/37800 [==============================] - 7s 198us/step - loss: 0.2110 - acc: 0.9404 - val_loss: 0.2100 - val_acc: 0.9429
Epoch 4/100
37800/37800 [==============================] - 7s 198us/step - loss: 0.1753 - acc: 0.9512 - val_loss: 0.1973 - val_acc: 0.9460
Epoch 5/100
37800/37800 [==============================] - 7s 194us/step - loss: 0.1512 - acc: 0.9569 - val_loss: 0.2638 - val_acc: 0.9293
Epoch 6/100
37800/37800 [==============================] - 8s 203us/step - loss: 0.1473 - acc: 0.9592 - val_loss: 0.1832 - val_acc: 0.9488
Epoch 7/100
37800/37800 [==============================] - 7s 198us/step - loss: 0.1301 - acc: 0.9635 - val_loss: 0.1

In [64]:
pred = model.predict_classes(test)
pred, len(pred)

(array([2, 0, 9, ..., 3, 9, 2]), 28000)

In [65]:
result = pd.DataFrame({'ImageId':list(range(1,test.shape[0]+1)),
                       'Label':pred})
result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [66]:
result.to_csv('output.csv', index=False, header=True)
client_242955d4e4ca4cad87468d854b73c857.upload_file(Filename='output.csv', Bucket='test-donotdelete-pr-ectconhjpferec', Key='output.csv')